# Imports

In [18]:
import numpy as np
import itertools
import copy

# Create Compatability Matrix

In [5]:
def generate_random_compatibility_matrix(num_clusters_method_a, num_clusters_method_b, constant_sum):
    random_matrix = np.zeros((num_clusters_method_a, num_clusters_method_b), dtype=int)
    cluster_a_list = np.zeros((num_clusters_method_a), dtype=int)

    for row_index in range(constant_sum):
        random_index = np.random.randint(0, num_clusters_method_a)
        cluster_a_list[random_index] += 1
    print("Cluster A sizes: ", cluster_a_list)

    for row_index in range(num_clusters_method_a):
        for _ in range(cluster_a_list[row_index]):
            random_index = np.random.randint(0, num_clusters_method_b)
            random_matrix[row_index, random_index] += 1

    return random_matrix


num_clusters_method_a = 4
num_clusters_method_b = 4
total_members = 100

compatibility_matrix = generate_random_compatibility_matrix(num_clusters_method_a, num_clusters_method_b, total_members)

print("Compatibility Matrix:\n", compatibility_matrix)


Cluster A sizes:  [30 29 19 22]
Compatibility Matrix:
 [[ 4  4  8 14]
 [ 5 12  8  4]
 [ 3  4  4  8]
 [ 7  8  2  5]]


# Brute Force Matching

In [14]:
def brute_force_matching(matrix):
    num_clusters_a = matrix.shape[0]

    all_permutations = list(itertools.permutations(range(num_clusters_a)))

    best_matching = None
    best_matching_sum = 0

    for permutation in all_permutations:
        matching_sum = np.sum(matrix[np.arange(num_clusters_a), permutation])
        if matching_sum > best_matching_sum:
            best_matching_sum = matching_sum
            best_matching = permutation

    return best_matching, best_matching_sum

In [39]:
best_matching, best_matching_sum = brute_force_matching(compatibility_matrix)
best_matching = [(index, number) for index, number in enumerate(best_matching)]
print("Best Matching: ", best_matching)
print("Best Matching Sum: ", best_matching_sum)

Best Matching:  [(0, 3), (1, 1), (2, 2), (3, 0)]
Best Matching Sum:  37


# Stable Matching Based Clustering Evaluation

In [35]:
def stable_matcher(matrix):

    def priorities(matrix):

        def rank(numbers):
            sorted_indices = np.argsort(numbers)[::-1]
            return sorted_indices
        
        output = []
        for comm1 in range(matrix.shape[0]):
            temp = []
            for comm2 in range(matrix.shape[1]):
                common = matrix[comm1,comm2]
                temp.append(common)
            output.append(rank(temp))
        return output

    def find_pair(List, value):
        for pair in List:
            if pair[1] == value:
                return pair[0]
            
    def find_index_of_value_in_array(arr, target_value):
        indices = np.where(arr == target_value)[0]
        if indices.size > 0:
            return indices[0]
        else:
            return 
            
    chooser = range(matrix.shape[0])
    wanted = range(matrix.shape[1])
    pairs = []
    unmatched_chooser = [index for index in range(len(chooser))]
    unmatched_wanted = [index for index in range(len(wanted))]
    prior_chooser = priorities(matrix)
    prior_wanted = priorities(matrix.T)
    while(len(unmatched_chooser)):
        comm1 = unmatched_chooser.pop()
        for matching_pred in prior_chooser[comm1]:
            if matching_pred in unmatched_wanted:
                pairs.append([comm1, matching_pred])
                unmatched_wanted.remove(matching_pred)
                break
            else:
                matched_with_wanted = find_pair(pairs,matching_pred)
                if find_index_of_value_in_array(prior_wanted[matching_pred], comm1) < find_index_of_value_in_array(prior_wanted[matching_pred], matched_with_wanted):
                    pairs.remove([matched_with_wanted, matching_pred])
                    pairs.append([comm1, matching_pred])
                    unmatched_chooser.append(matched_with_wanted)
                    break
    return pairs

In [41]:
pairs = stable_matcher(compatibility_matrix)

sorted_pairs = sorted(pairs, key=lambda x: x[0])
best_score =  np.sum(compatibility_matrix[[pair[0] for pair in sorted_pairs], [pair[1] for pair in sorted_pairs]])

print("Best Matching Based on Stable Matching: ", sorted_pairs)
print("Best Matching Sum Based on Stable Matching: ", best_score)

Best Matching Based on Stable Matching:  [[0, 3], [1, 1], [2, 2], [3, 0]]
Best Matching Sum Based on Stable Matching:  37
